# Compute reference solutions

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import h5py
import pestoseis.ttimerays as tr

### Rays horizontally layered medium

In [3]:
# number of layers
Nlay = 120
# depth of layers -- includes both top and bottom (Nlay+1)
laydepth = np.linspace(0.0,2000.0,Nlay+1)[1:]
# velocity
velmod = np.linspace(2000.0,3000.0,Nlay)
# origin of ray
xystart = np.array([0.0, 0.0])
# take off angle
takeoffangle = 45.0

# trace a single ray
raypath,tt,dist = tr.tracerayhorlay(laydepth, velmod, xystart, takeoffangle)

 timeray(): take off angle: 45.0


In [8]:
np.save("raypath_horizLayeredModel.npy",raypath) 
np.save("tt_horizLayeredModel.npy",tt) 
np.save("dist_horizLayeredModel.npy",dist)

### Linear grad velocity

In [9]:
def analyticalsollingrad2D(gridpar,xsrcpos,ysrcpos):
    ##################################
    ## linear gradient of velocity

    ## source must be *on* the top surface
    #@show ysrcpos
    assert (ysrcpos == 0.0)

    # position of the grid nodes
    xgridpos = np.array([i * gridpar["dh"] + gridpar["xinit"] for i in range(gridpar["nx"])])
    ygridpos = np.array([i * gridpar["dh"] + gridpar["yinit"] for i in range(gridpar["ny"])])
    vel0 = 2.0
    # gradient of velociy
    gr = 0.05
    ## construct the 2D velocity model
    velmod = np.zeros([gridpar["nx"],gridpar["ny"]]) 
    for i in range(gridpar["nx"]):
        velmod[i,:] = vel0 + gr * (ygridpos - ysrcpos)

    # https://pubs.geoscienceworld.org/books/book/1011/lessons-in-seismic-computing
    # Lesson No. 41: Linear Distribution of Velocity—V. The Wave-Fronts 

    ## Analytic solution
    ansol2d  = np.zeros([gridpar["nx"],gridpar["ny"]])
    for j in range(gridpar["ny"]):
        for i in range(gridpar["nx"]):
            x = xgridpos[i]-xsrcpos
            h = ygridpos[j]-ysrcpos
            ansol2d[i,j] = 1 / gr * np.arccosh( 1 + (gr ** 2 * (x ** 2 + h ** 2)) / (2 * vel0 * velmod[i,j]))
    return ansol2d,velmod
        
nx = 100
ny = 100
h = 100
xinit = 0.0
yinit = 0.0
hgrid = nx / h
gridpar = tr.setupgrid(nx,ny,hgrid,xinit,yinit)
nrec = 100
x_rec = np.linspace(0.0,gridpar["nx"],nrec)
y_rec = np.zeros(nrec)
receivers = np.vstack((x_rec,y_rec))

# one source on top of domain
nsrc = 1
xsrcpos = 20.0
ysrcpos = 0.0
srccoo = np.zeros([nsrc,2])
srccoo[:,0] = xsrcpos;
srccoo[:,1] = ysrcpos;

ansol2d,velmod_analytical = analyticalsollingrad2D(gridpar, xsrcpos, ysrcpos)
ttpick,ttime = tr.traveltime(velmod_analytical, gridpar, srccoo, receivers)

Calculating traveltime for source 1 of 1 


In [ ]:
diff = np.abs(ansol2d - ttime[0][:-1,:-1]) 

In [ ]:
np.save("diff_analytical_numerical_linearGradVelocity.npy", )

### Test rays homogeneous model

In [2]:
# create a 2D grid with 50x30 cells
nx,ny = 100,60
# size of the cells
dh = 2.5
# origin of grid axes
xinit,yinit = 0.0,0.0

# create the dictionary containing the grid parameters
gridpar = tr.setupgrid(nx, ny, dh, xinit, yinit)

# define a velocity model
velmod = np.zeros((nx,ny)) + 1500

# define the position of sources and receivers, e.g.,
recs = np.array([[30.4, 22.3],
                 [10.1,  20.0],
                 [12.4,  9.5]])
srcs = np.array([[ 3.4,  2.3],
                 [42.4, 15.5]])
## calculate all traveltimes
ttpick,ttime = tr.traveltime(velmod,gridpar,srcs,recs)

## now trace rays (ttime contains a set of 2D traveltime arrays)
rays = tr.traceallrays(gridpar,srcs,recs,ttime)

rays_xy = np.zeros([rays.shape[0],rays[0][0]["xy"].shape[0],rays[0][0]["xy"].shape[1]])
rays_ij = np.zeros([rays.shape[0],rays[0][0]["ij"].shape[0],rays[0][0]["ij"].shape[1]])
rays_segment_len = np.zeros([rays.shape[0],rays[0][0]["segment_len"].shape[0],rays[0][0]["segment_len"].shape[1]])
for i in range(rays.shape[0]):
    rays_xy[i,:] = rays[0][0]["xy"]
    rays_ij[i,:] = rays[0][0]["ij"] 
    rays_segment_len[i,:] = rays[0][0]["segment_len"] 

Calculating traveltime for source 2 of 2 
tracing rays for source 2 of 2     


In [3]:
np.save("rays_homogeneous.npy",rays,allow_pickle=True)